In [ ]:
from pathlib import Path

import ee
import eemont  # noqa: F401
import geemap
import geemap.colormaps as cm

from bisonlab.data import s2_sr_harmonized
from bisonlab.io import kml_to_geodataframe
from bisonlab.utils import mask_exclude, mask_include

In [ ]:
ee.Authenticate()
ee.Initialize()

## Load Parcels and Masks

In [ ]:
filepath = Path().cwd().parent.parent / "data" / "local" / "WRTBI-Morton Soil Types.kml"

df = kml_to_geodataframe(filepath)

df = df.rename(columns={"Name": "subsection", "layer": "parcel"})
df = df.drop(columns="Description")

In [ ]:
# Union all "mask" polygons and create a separate mask dataframe
idx = df["parcel"] == "mask"
df_mask = df[idx].dissolve(by="parcel").reset_index()

# Drop "mask" from parcel dataframe
df = df.drop(index=idx.index[idx])

In [ ]:
# Convert parcels dataframe to ee.featureCollection
parcels = geemap.geopandas_to_ee(df)
mask = geemap.geopandas_to_ee(df_mask)

In [ ]:
# Plot parcels and Mask polygons
m = geemap.Map()
m.addLayer(parcels, {"color": "red"}, "Parcels")
m.addLayer(mask, {"color": "blue"}, "mask")
m.centerObject(mask, 17)
m

## Mask single image

In [ ]:
# Get 1 image of NDVI
img = (
    s2_sr_harmonized(parcels, "2022-08-07", "2022-08-08")
    .spectralIndices("NDVI")
    .select("NDVI")
    .first()
)

### Exclude region

In [ ]:
masked_out = mask_exclude(img, mask)

In [ ]:
m = geemap.Map()
m.add_basemap(basemap="SATELLITE")
m.addLayer(masked_out, {"min": -1, "max": 1, "palette": cm.palettes.ndvi}, "NDVI")
m.centerObject(mask, 17)
m

### Include regions

In [ ]:
masked_in = mask_include(img, mask)

In [ ]:
m = geemap.Map()
m.add_basemap(basemap="SATELLITE")
m.addLayer(masked_in, {"min": -1, "max": 1, "palette": cm.palettes.ndvi}, "NDVI")
m.centerObject(mask, 17)
m

## Mask time series

In [ ]:
# Get timeseries image of NDVI - 10 images in ImageCollection

start_date = "2022-08-01"
end_date = "2022-08-31"

ndvi = (
    s2_sr_harmonized(parcels, start_date, end_date)
    .spectralIndices("NDVI")
    .select("NDVI")
)

In [ ]:
# Apply mask to each image in the collection
masked = ndvi.map(lambda img: mask_exclude(img, mask))

In [ ]:
# Choose first image in collection
img = masked.first()

In [ ]:
m = geemap.Map()
m.add_basemap(basemap="SATELLITE")
m.addLayer(img, {"min": -1, "max": 1, "palette": cm.palettes.ndvi}, "NDVI")
m.centerObject(mask, 17)
m